# Creating the model inferencing function

## Initialize nuclio emulation, environment variables and configuration
use `# nuclio: ignore` for sections that dont need to be copied to the function

In [2]:
# nuclio: ignore
import nuclio

In [3]:
# nuclio: ignore
from os import environ
version = '1.0'
environ['MODEL_FILE'] = 'netops.v{}.model'.format(version)

In [4]:
%nuclio env -c V3IO_USERNAME=${V3IO_USERNAME}
%nuclio env -c V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}
%nuclio env -c MODEL_FILE=${MODEL_FILE}
%nuclio env -c MODEL_FILEPATH=/tmp/mlmodel/${MODEL_FILE}
%nuclio env -l MODEL_FILEPATH=models/${MODEL_FILE}

%nuclio: setting 'MODEL_FILEPATH' environment variable


### Install requiered packages
`%nuclio cmd` allows you to run image build instructions and install packages<br>
Note: when using the `-c` option commands will only run in nuclio, not locally

In [5]:
%%nuclio cmd -c
pip install pandas
pip install sklearn
pip install v3io_frames
pip install dask_ml
apt-get update && apt-get install -y wget
mkdir -p /tmp/mlmodel

In [6]:
# copy the model file into the function
%nuclio cmd -c wget -O /tmp/mlmodel/${MODEL_FILE} --header "x-v3io-session-key: ${V3IO_ACCESS_KEY}" http://${V3IO_WEBAPI_SERVICE_HOST}:8081/users/${V3IO_USERNAME}/demos/netops/models/${MODEL_FILE}

## Set a cron trigger with 1 minute interval

In [7]:
%%nuclio config 
spec.triggers.secs.kind = "cron"
spec.triggers.secs.attributes.interval = "600s"
spec.build.baseImage = "python:3.6-jessie"

%nuclio: setting spec.triggers.secs.kind to 'cron'
%nuclio: setting spec.triggers.secs.attributes.interval to '600s'
%nuclio: setting spec.build.baseImage to 'python:3.6-jessie'


## Define function init

In [8]:
import pandas as pd
import pickle
import sklearn
from dask_ml.wrappers import ParallelPostFit
import v3io_frames as v3f
from os import environ

# Define v3io TSDB client
client = v3f.Client('framesd:8081')

# Load model
with open(environ['MODEL_FILEPATH'], 'rb+') as f:
    model = pickle.load(f) 
    
clf = ParallelPostFit(estimator=model,
                      scoring='accuracy')

FileNotFoundError: [Errno 2] No such file or directory: 'models/netops.v1.0.model'

In [ ]:
%%nuclio handler
# Load last hour data
df = client.read(backend='tsdb', query='select avg(cpu_utilization) as cpu_utilization, avg(latency) as latency, avg(packet_loss) as packet_loss, avg(throughput) as throughput from netops_metrics_jupyter group by company, data_center, device',
            start="now-1d", end='now+1d', multi_index=True, step='1h')
df['prediction'] = clf.predict(df)
#df.reset_index(inplace=True)
#client.write(backend='tsdb', table='netops_predictions', dfs=[df], index_cols=['company', 'data_center', 'device', 'time'])
str(df['prediction']) # nuclio: return

In [ ]:
%nuclio deploy -n netops-infer -p netops -c